# This is just a notebook to visualise 1kHz filtered raw data

In [ ]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
import fnmatch
import os
import mmap
import xarray as xr
import dask.array as da
%matplotlib widget
from IPython.display import display
from ipyfilechooser import FileChooser

from ephyviewer import mkQApp, MainViewer, TraceViewer

In [ ]:
def other_mmap(filename, shape, dtype, chunks):
    with open(filename, "rb") as fh:
        # `mmap` duplicates the file descriptor
        # `0` means map the full file
        mm = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)

    # coerce to NumPy array of expected type and shape
    a = np.asarray(mm).view(dtype).reshape(shape)

    # Don't call `asarray`
    # name=False to avoid hashing
    a_d = da.from_array(a, chunks=chunks, asarray=False, name=False)
    return a_d

# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

In [22]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"


# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>OpenEphys Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023\Gaelle\Baseline_recordi…

Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)


Load LFPs data

In [25]:
folderpath = Path(dpath)
numchannels=32

if find_files_with_string(folderpath,  ".bin"): #Bonsai
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype=np.uint16)
    All = All - int(65535/2)
    All = All.astype(np.int16)
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: OE32channels.bin,', numchannels, 'channels')

elif find_files_with_string(folderpath,  "continuous.dat"): #OpenEphys
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype=np.int16)

    #All =np.memmap(matching_file, dtype='int16', mode='c')
    
    #All = All - int(65535/2)
    #All = All.astype(np.int16)    
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: continuous.dat,', numchannels, 'channels')

elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): #OpenEphys Gaelle's Data
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    
    #All = All.reshape(-1,numchannels)
    print('File loaded: RawDataChannelExtractedDS.npy,', numchannels, 'channels')

All.shape

File loaded: RawDataChannelExtractedDS.npy, 32 channels


(940555, 31)

In [ ]:
All = np.fromfile(matching_file, dtype=np.int16)
numchannels=64
All = All.reshape(-1,numchannels)

In [ ]:
#BOB
S1=All[:, 5]-All[:,6]
PFC=All[:, 2]-All[:,3]
CA1=All[:, 29]-All[:,30]
EMG=All[:,27]
combined = np.stack([S1, PFC, CA1, EMG/10], axis = 1)

In [ ]:
# Filtre parameter:
f_lowcut = 1
f_hicut = 50.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')


In [ ]:
#Blue
PFC=All[:, 14]-All[:,15]
PFC = signal.filtfilt(b, a, PFC)
S1=All[:, 8]-All[:,9]
S1 = signal.filtfilt(b, a, S1)
EMG=All[:,20]
combined = np.stack([S1,PFC, EMG/10], axis = 1)

In [ ]:
#Pink
PFC=All[:, 26+32]-All[:,27+32]
EMG=All[:,9+32]
S1=All[:, 0+32]-All[:,1+32]
combined = np.stack([S1,PFC, EMG/10], axis = 1)

Visualise All

In [26]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers

win = MainViewer()
view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')
#add them to mainwindow

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

win.add_view(view1)


#Run
win.show()
app.exec_()


0

In [ ]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers

win = MainViewer()
#view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')

source = InMemoryAnalogSignalSource(combined, sample_rate, t_start)
view1 = TraceViewer(source=source, name='trace')

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#create a time freq viewer conencted to the same source
view2 = TimeFreqViewer(source=source, name='tfr')

view2.params['show_axis'] = True
view2.params['timefreq', 'deltafreq'] = 1
view2.params['timefreq', 'f_stop'] = 200
#view2.by_channel_params['ch0', 'timefreq', 'f_stop'] = 20
#view2.by_channel_params['ch1', 'timefreq', 'f_stop'] = 150
view2.by_channel_params['ch0', 'visible'] = False
view2.by_channel_params['ch1', 'visible'] = True
view2.by_channel_params['ch1', 'clim'] = 100

#add them to mainwindow
win.add_view(view1)
win.add_view(view2)


#Run
win.show()
app.exec_()
